# Этап 1: Сбор метаданных

**Файл:** `metadata_parse.ipynb`

Собираем метаданные всех статей через arXiv API

## Особенности реализации

- **Помесячный сбор** — обход лимитов
- **Фильтрация по primary_category** — исключаем cross-listed статьи
- **Rate limiting** — задержка 3 секунды между запросами
- **Retry logic** — 3 попытки при ошибках

---

In [13]:
import arxiv
import pandas as pd
from pathlib import Path
from tqdm import tqdm
from datetime import datetime, timedelta

In [2]:
# ===== ПАРАМЕТРЫ =====
N = 100_000 # Максимум статей в поиске, после фильтрации станет меньше
CAT = 'cs.CL'
YEAR = 2025

In [3]:
# ===== КЛИЕНТ =====
client = arxiv.Client(
    page_size=1000, # У arXiv API лимит - 2000. Выше него не ставить.
    delay_seconds=3,
    num_retries=3
)

In [4]:
%%time
# ===== ГЛАВНЫЙ ЦИКЛ СБОРА =====
print(f"🔍 Сбор статей cs.CL за {YEAR} год\n")

all_metadata = []

for month in range(1, 13):
    # Первый день месяца
    start = datetime(YEAR, month, 1)
    
    # Последний день месяца
    if month == 12:
        end = datetime(YEAR, 12, 31)
    else:
        end = datetime(YEAR, month + 1, 1) - timedelta(days=1)
    
    start_str = start.strftime('%Y%m%d')
    end_str = end.strftime('%Y%m%d')
    
    # Query для arXiv
    query = f"cat:{CAT} AND submittedDate:[{start_str} TO {end_str}]"
    
    search = arxiv.Search(
        query=query,
        sort_by=arxiv.SortCriterion.SubmittedDate,
        max_results=50000
    )
    
    print(f"📅 Месяц {month:02d}: ", end='', flush=True)
    
    # Собираем статьи
    month_papers = []
    try:
        for paper in client.results(search):
            month_papers.append({
                'arxiv_id': paper.get_short_id(),
                'title': paper.title,
                'authors': ', '.join([a.name for a in paper.authors]),
                'summary': paper.summary,
                'primary_category': paper.primary_category,
                'categories': ', '.join(paper.categories),
                'published': paper.published.strftime('%Y-%m-%d'),
                'updated': paper.updated.strftime('%Y-%m-%d'),
                'pdf_url': paper.pdf_url,
                'entry_id': paper.entry_id,
            })
    except Exception as e:
        print(f"❌ Ошибка: {e}")
        continue
    
    all_metadata.extend(month_papers)
    print(f"{len(month_papers)} статей")

print(f"\nВсего собрано: {len(all_metadata)} статей")

🔍 Сбор статей cs.CL за 2025 год

📅 Месяц 01: 1313 статей
📅 Месяц 02: 2476 статей
📅 Месяц 03: 1878 статей
📅 Месяц 04: 1605 статей
📅 Месяц 05: 2953 статей
📅 Месяц 06: 2323 статей
📅 Месяц 07: 1639 статей
📅 Месяц 08: 1830 статей
📅 Месяц 09: 2232 статей
📅 Месяц 10: 2587 статей
📅 Месяц 11: 1547 статей
📅 Месяц 12: 508 статей

✅ Всего собрано: 22891 статей
CPU times: total: 21.6 s
Wall time: 2min 27s


In [5]:
# ===== СОЗДАЕМ DATAFRAME =====
df = pd.DataFrame(all_metadata)

In [6]:
# ===== ФИЛЬТРУЕМ ПО PRIMARY CATEGORY =====
df_filtered = df[df['primary_category'] == CAT].copy()

print(f"\n=====СТАТИСТИКА=====")
print(f"Всего найдено: {len(df)}")
print(f"С primary category {CAT}: {len(df_filtered)}")


=====СТАТИСТИКА=====
Всего найдено: 22891
С primary category cs.CL: 16503


In [7]:
# ===== СОХРАНЯЕМ =====
output_dir = Path('data/metadata')
output_dir.mkdir(parents=True, exist_ok=True)

df_filtered.to_csv('data/metadata/papers_metadata.csv', index=False)

print(f"\nМетаданные сохранены:")
print(f"   - data/metadata/papers_metadata.csv")


✅ Метаданные сохранены:
   - data/metadata/papers_metadata.csv
   - data/metadata/papers_metadata.json


In [14]:
# ===== ОБЗОР =====
df = pd.read_csv('data/metadata/papers_metadata.csv')

In [16]:
df.sample(5)

,arxiv_id,title,authors,summary,primary_category,categories,published,updated,pdf_url,entry_id
11846,2509.23550v1,Automatic Speech Recognition for Greek Medical...,"Vardis Georgilas, Themos Stafylakis",Medical dictation systems are essential tools ...,cs.CL,"cs.CL, cs.AI",2025-09-28,2025-09-28,https://arxiv.org/pdf/2509.23550v1,http://arxiv.org/abs/2509.23550v1
13236,2510.26974v1,Overview of the MEDIQA-OE 2025 Shared Task on ...,"Jean-Philippe Corbeil, Asma Ben Abacha, Jerome...",Clinical documentation increasingly uses autom...,cs.CL,"cs.CL, cs.AI",2025-10-30,2025-10-30,https://arxiv.org/pdf/2510.26974v1,http://arxiv.org/abs/2510.26974v1
12241,2509.18344v2,Speculate Deep and Accurate: Lossless and Trai...,"Pei-Shuo Wang, Jian-Jia Chen, Chun-Che Yang, C...",The immense model sizes of large language mode...,cs.CL,cs.CL,2025-09-22,2025-10-08,https://arxiv.org/pdf/2509.18344v2,http://arxiv.org/abs/2509.18344v2
3608,2503.07550v1,KSOD: Knowledge Supplement for LLMs On Demand,"Haoran Li, Junfeng Hu",Large Language Models (LLMs) have demonstrated...,cs.CL,"cs.CL, cs.AI, cs.LG",2025-03-10,2025-03-10,https://arxiv.org/pdf/2503.07550v1,http://arxiv.org/abs/2503.07550v1
11414,2508.10004v1,User Perception of Attention Visualizations: E...,"Andrés Carvallo, Denis Parra, Peter Brusilovsk...",The attention mechanism is a core component of...,cs.CL,"cs.CL, cs.AI, cs.HC, cs.IR, cs.LG",2025-08-05,2025-08-05,https://arxiv.org/pdf/2508.10004v1,http://arxiv.org/abs/2508.10004v1
